http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf

In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import time
import cmath
import sys
import random
from mnist import MNIST

In [2]:
def loadMNIST():
    mndata = MNIST('../MNIST')
    imagesTest, labelsTest = mndata.load_training()
    imagesTrain, labelsTrain = mndata.load_testing()
    return imagesTrain, labelsTrain, imagesTest, labelsTest

In [3]:
def convolve(im, kernel, bias=0):
    kernelSize = len(kernel)
    ret = (np.zeros((len(im) - 2*(kernelSize//2), len(im[0]) - 2*(kernelSize//2))))
    for i in range(kernelSize//2, len(im) - kernelSize//2):
        for j in range(kernelSize//2, len(im[0]) - kernelSize//2):
            for k in range(kernelSize):
                for l in range(kernelSize):
                    ret[i - kernelSize//2][j - kernelSize//2] += im[i - kernelSize//2 + k][j - kernelSize//2 + l]*kernel[k][l]
    ret = ret + bias
    return ret

In [4]:
def avgPool(im, k):
    ret = np.zeros(((len(im)//2), len(im[0])//2))
    for i in range(len(ret)):
        for j in range(len(ret[0])):
            for p in range(k):
                for l in range(k):
                    ret[i][j] += im[i*2 + p][j*2 + l]/(k**2)
    return ret

In [5]:
def reshape(x):
    return np.uint8(np.reshape(x, (int(math.sqrt(len(x))), int(math.sqrt(len(x))))))

In [6]:
def preprocess(im):
    return np.uint8(np.pad(im, (2, 2)))

In [7]:
xTrain, yTrain, xTest, yTest = loadMNIST()

In [8]:
for i in range(len(xTrain)):
    xTrain[i] = preprocess(reshape(xTrain[i]))
for i in range(len(xTest)):
    xTest[i] = preprocess(reshape(xTest[i]))

## Parameters
https://cdn.analyticsvidhya.com/wp-content/uploads/2021/03/Screenshot-from-2021-03-18-12-56-51.png

In [9]:
conv = [(np.random.rand(6, 5, 5)), (np.random.rand(6, 16, 5, 5)), (np.random.rand(16, 120, 5, 5))]
bConv = [(np.random.rand(6)), (np.random.rand(16)), (np.random.rand(120))]
w1 = np.random.rand(120, 84)
b1 = np.zeros(84)
w2 = np.random.rand(84, 10)
b2 = np.zeros(10)

In [10]:
def forward(im):
    # Convolution 1
    ret1 = []
    for i in range(len(conv[0])):
        ret1.append(np.tanh(convolve(im, conv[0][i], bConv[0][i])))
        
    # Average pooling
    for i in range(len(ret1)):
        ret1[i] = avgPool(ret1[i], 2)
        
    # Convolution 2
    ret2 = []
    for i in range(len(conv[1][0])):
        tmp = np.zeros((len(ret1[0]) - 2*(len(conv[1][0][i])//2), len(ret1[0][0]) - 2*(len(conv[1][0][i])//2)))
        for j in range(len(conv[1])):
            tmp += convolve(ret1[j], conv[1][j][i])
        tmp +=  bConv[1][i]
        ret2.append(np.tanh(tmp))
        
    # Average pooling
    for i in range(len(ret2)):
        ret2[i] = avgPool(ret2[i], 2)
        
    # Convolution 3
    ret3 = []
    for i in range(len(conv[2][0])):
        tmp = np.zeros((len(ret2[0]) - 2*(len(conv[2][0][i])//2), len(ret2[0][0]) - 2*(len(conv[2][0][i])//2)))
        for j in range(len(conv[2])):
            tmp += convolve(ret2[j], conv[2][j][i])
        tmp +=  bConv[2][i]
        ret3.append(np.tanh(tmp))
    
    # Fully connected 1
    ret4 = []
    for i in range(len(w1[0])):
        tmp = 0
        for j in range(len(w1)):
            tmp += w1[j][i]*ret3[j]
        tmp += b1[i]
        ret4.append(np.tanh(tmp))

    # Fully connected 2
    ret5 = []
    for i in range(len(w2[0])):
        tmp = 0
        for j in range(len(w2)):
            tmp += w2[j][i]*ret4[j]
        tmp += b2[i]
        ret5.append(tmp)
    
    # Softmax activation
    expSum = 0
    for i in range(len(ret5)):
        expSum += math.exp(ret5[i])
    ret5 = np.exp(ret5)/expSum
    
    return ret5

In [11]:
forward(xTrain[0])

array([[[2.96914436e-05]],

       [[6.61820551e-05]],

       [[1.10825498e-04]],

       [[2.73965546e-06]],

       [[9.34246358e-01]],

       [[9.00369244e-05]],

       [[3.78124723e-04]],

       [[6.47432241e-02]],

       [[1.70825249e-04]],

       [[1.61992509e-04]]])